# 10 ドキュメント型DB（MongoDB）

- **[10.1 NoSQLとRDBMS](#10.1-NoSQLとRDBMS)**
    - [10.1.1 NoSQLとは](#10.1.1-NoSQLとは)
    - [10.1.2 NoSQLとRDBMSの違い](#10.1.2-NoSQLとRDBMSの違い)
<br><br>
- **[10.2 MongoDBの基礎知識](#10.2-MongoDBの基礎知識)** 
    - [10.2.1 MongoDBの概要と特徴](#10.2.1-MongoDBの概要と特徴)
    - [10.2.2 MongoDBの仕組み](#10.2.2-MongoDBの仕組み)
    - [10.2.3 MongoDB使用環境](#10.2.3-MongoDB使用環境)
    - [10.2.4 PyMongoの基本操作](#10.2.4-PyMongoの基本操作)
    - [10.2.5 MongoDBのデータ型](#10.2.5-MongoDBのデータ型)
<br><br> 
- **[10.3 MongoDBの基本操作](#10.3-MongoDBの基本操作)** 
    - [10.3.1 データの挿入](#10.3.1-データの挿入-&#40;Create&#41;)
    - [10.3.2 データの検索](#10.3.2-データの検索-&#40;Read&#41;)
        - [10.3.2.1 カーソル操作](#10.3.2.1-カーソル操作)
    - [10.3.2 データの更新](#10.3.3-データの更新-&#40;Update&#41;)
    - [10.3.4 データの削除](#10.3.4-データの削除-&#40;Delete&#41;)
    - [10.3.5 データの集計](#10.3.5-データの集計)
<br><br>
- **[10.4 MongoDBのパフォーマンス向上](#10.4-MongoDBのパフォーマンス向上)**
    - [10.4.1 インデックス](#10.4.1-インデックス)
    - [10.4.2 MongoDBの統計・メットリックス](#10.4.2-MongoDBの統計・メットリックス)
    - [10.4.3 データのバックアップ・エクスポート](#10.4.3-データのバックアップ・エクスポート)
<br><br>
- **[10.5 総合問題](#10.5-総合問題)**
    - [10.5.1 基本操作の復習](#10.5.1-基本操作の復習)
    - [10.5.2 Geospatialインデックスの応用](#10.5.2-Geospatialインデックスの応用)

***

## 10.1 NoSQLとRDBMS

### 10.1.1 NoSQLとは

前章までは、従来のリレーショナルデータベースについて学びました。
本章では、いわゆるNoSQL系のデータベースの一つであるMongoDBについて学びます。

従来のRDBMSが明確なデータ構造を定義し、SQL言語による優れた検索性能を持ち、強力なトランザクション機能を備えています。しかしその反面、水平スケーリングしづらく、マスタスレーブ構造のため書き込み負荷が集中しやすく、耐障害性も上げにくいことなどの弱点があります。近年、それらの弱点に対処するのに、生まれたのがNoSQL系データベースです。

NoSQLとは「Not + Only + SQL」の頭文字を取ったもので、一般的にすべての非リレーショナルデータベースを指す単語です。NoSQL系データベースには、ドキュメント型（例:[MongoDB](https://www.mongodb.com/)）、グラフ型（例:[Neo4j](https://neo4j.com/)）、キー値型（例:[Amazon DynamoDB](https://aws.amazon.com/dynamodb/)）、列指向型(例:[Cassandra](http://cassandra.apache.org/)）など、様々なデータモデルが利用されています。

### 10.1.2 NoSQLとRDBMSの違い

前節で触れたスケーラビリティやデータモデルの違いの他に多くの違いがあります。

NoSQLとのRDBMSとの間特に重要な違いの一つとして、データの取得・操作方法が挙げられます。RDBMSでは構造化クエリ言語 (SQL) 準拠のクエリを使用するのに対して、NoSQLの多くはオブジェクト指向のAPIを使用して、データの挿入・検索等を簡単に行うことができます（中にはSQLに似たクエリ言語をサポートするものもあります）。すなわち、NoSQLではデータベースおよび実際のアプリケーションのデータ構造が似ており、RDBMSより扱いやすいことが多いです。

また、NoSQLはRDBMSと違って、データを挿入する前に事前にスキーマを定義・固定する必要がないことが多いです。自由に様々な形式のデータを一緒に保存することが可能で、RDBMSよりフレキシブルです。逆に、NoSQLの弱点の一つはトランザクションの機能がないことです。このあと勉強するMongoDBではトランザクション相当の機能を実現できなくもないですが、初心者には少し高度なトピックなので、詳細は割愛します。

***

## 10.2 MongoDBの基礎知識

### 10.2.1 MongoDBの概要と特徴

MongoDBとは現在最も広く利用されているオープンソースのドキュメント型NoSQLデータベースです。従来のRDBMSで使われるテーブル構造ではなく、動的なスキーマを持ったJSONのようなドキュメント（BSON形式）を用いています。
固定スキーマのないBSONを使うことで、一つの記録で複雑な階層関係を持つデータを格納できたり、必要に応じてフィールドを追加または削除することなどが簡単にできます。そのため、JSON形式を使う多くのWeb APIとの相性がよく、開発がより迅速になると感じる利用者が多いです。

また、MongoDBの豊富なインデックスとデータ構造を使うことで、RDBMSではできない様々なことができます。MongoDBの様々なインデックスを使うことで、位置情報の取扱いを容易にしたり、文字列の検索性能を向上したり、古いデータを自動的に削除したりできます。また、MongoDBの様々なデータ型を使うことで、階層構造のデータを無理に正規化しなくて済みますし、BSONのバイナリ型を使うことで特殊なデータベースシステムを作ることまでできます（例: [GridFS](https://docs.mongodb.com/manual/core/gridfs/)、[Arctic](https://github.com/manahl/arctic)）。

つまり、どちらがより向いているかは最終的には扱っているデータの性質に依りますが、MongoDBはRDBMSよりフレキシブルでパフォーマンスも高いことから、RDBMSが使われているほとんどの場面において代わりにMongoDBを問題なく使用することが可能です。

### 10.2.2 MongoDBの仕組み

それでは、具体的なコマンド等を説明する前に、MongoDBの動作の基本的な概念および仕組みを学びましょう。

* 一つのMongoDBのインスタンスの中には複数の**データベース**を管理することが可能です。ここでいうデータベースとは前章までで学んだMySQLと同様にMongoDBにおけるもっとも高レベルなコンテナのことです。
* 各データベースは一般的に複数の**コレクション**からなります。コレクションはMySQLでいうテーブルと似たような概念です。
* 各コレクションは一般的に複数の**ドキュメント**からなります。MySQLでいう**行**と似たような概念です。また、すべてのドキュメントにはユニークな`_id`フールドが与えられます。
* 各ドキュメントは1つ以上の**フィールド**からなります。フィールドはキーと値の対からなります。MySQLでいう**列**に近い概念です。
* MongoDBの**インデックス**機能はMySQLのインデックスとよく似ており、インデックスを作ることで、検索・ソート性能を向上させることが可能です。
* MongoDBの**カーソル**もMySQLのものとよく似ており、検索したデータへのポインタのようなものです。MongoDBからデータ取得しようとするときに返されるのは実際のデータではなく、このカーソルであることをと覚えておいてください。

MongoDBでは「テーブル」を「コレクション」と呼ぶなど、一見不必要な新しい用語が登場しますが、これらは実は似て非なるものであることを頭の片隅に入れておいてください。MySQLでは**列**が**テーブル**ごとに定義されるのに対して、MongoDBのようなドキュメント型DBMSでは**フィールド**が**ドキュメント**ごとに定義されている、というのがもっとも根本的な違いです。すなわち、コレクション内の各ドキュメントは独立しており、自由に自身のフィールドを定義することができます。 そのため、**コレクション**は**テーブル**よりシンプルなコンテナでありながら、**ドキュメント**は通常、**行**に比べてより多くの情報を保有します。

### 10.2.3 MongoDB使用環境

MongoDBを使うには(1)**MongoDBシェル**と呼ばれるCLIツールを利用する、もしくは(2)自分の好きなプログラミング言語とMongoDBをつなぐ**MongoDBドライバ**と呼ばれるライブリを利用します。実際のアプリケーション開発をするときは、基本的にドライバから使うことになりますが、ちょっとしたadminタスク等にはMongoシェルが便利です。また、MongoDBユーザの共通語がシェルで使われるJavascriptなので、本・ネット上のリソースでは基本的にMongoシェル/Javascriptを使った例が他の言語に比べて多いです。

**Mongoシェル**はMongoDBへのインタラクティブなJavaScriptインターフェイスで、データの挿入・検索・更新・削除・集約等の操作の実行ができます。Mongoシェルは、MongoDB本体と一緒にインストールされるので、MongoDBが使える環境であれば、Mongoシェルをコマンドライン<sup id="a1">[1](#f1)</sup>
から`mongo`を実行して使うことができます。

多くの言語用の**MongoDBドライバ**が[開発されています](https://docs.mongodb.com/ecosystem/drivers/)が、本講座ではPythonを前提としているため、Python用のMongoDBドライバである[pymongo](https://github.com/mongodb/mongo-python-driver)を中心にMongoDBのコマンド等を紹介していきます。

> <sup id="f1">**[1]** CLIが苦手な方には様々な[GUIツール](https://docs.mongodb.com/ecosystem/tools/administration-interfaces/)もあります[↩](#a1)</sup>

>#### MongoシェルJupyterから実行
>JupyterではPythonだけでなく、様々なプログラミング言語を実行することが可能です。Jupyterと言語を結びつけるものを**カーネル**といいますが、MongoDB用のJupyterカーネルには拙作の[iMongo](https://github.com/gusutabopb/imongo)というのがあります。気になる方はぜひ使ってみたください。

#### MongoDBのインストール

MongoDBはLinux、macOS、Windowsで動作します。インストールの手順はOSごとに異なるので、詳細は公式ドキュメンテーションを参考にしてください:

- [Linuxでのインストール手順](https://docs.mongodb.com/manual/administration/install-on-linux/)
- [macOSでのインストール手順](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-os-x/)
- [Windowsでのインストール手順](https://docs.mongodb.com/manual/tutorial/install-mongodb-on-windows/)

#### iLectでの環境準備

iLectではすでにMongoDBがインストールされていますが、MongoDBを立ち上げるには下記の手順に従ってください:

1. Jupyter Labからターミナルを立ち上げ、`mkdir /root/userspace/mongod`を実行
2. ターミナルから`mongod`を実行
3. `mongod`を実行したままにし、Notebookに戻る

ちなみに、`mongod`実行時は下記のような出力がでれば正常です:

```
root@notebook:~# mongod
2018-01-29T02:57:39.280+0000 I CONTROL  [initandlisten] MongoDB starting : pid=75 port=27017 dbpath=/data/db 64-bit host=notebook
2018-01-29T02:57:39.280+0000 I CONTROL  [initandlisten] db version v3.6.2
2018-01-29T02:57:39.280+0000 I CONTROL  [initandlisten] git version: 489d177dbd0f0420a8ca04d39fd78d0a2c539420
2018-01-29T02:57:39.280+0000 I CONTROL  [initandlisten] OpenSSL version: OpenSSL 1.0.1t  3 May 2016
2018-01-29T02:57:39.280+0000 I CONTROL  [initandlisten] allocator: tcmalloc
```

また、MongoDBを再起動するためには、`mongod`が回っているターミナルから`C

#### PyMongoのインストール

もしご使用の環境でPyMongoがインストールされていなければ、`pip`からインストールしてください:

``
$ pip install pymongo
``

上記のコマンドを実行すると、`pymongo`というパッケージ以外に、[`bson`](https://api.mongodb.com/python/3.4.0/api/bson/index.html)と[`gridfs`](https://api.mongodb.com/python/3.4.0/api/gridfs/index.html)というパッケージが一緒にインストールされます。前者はBSONオブジェクトを扱うためのツール、後者はGridFSを使うためのものです。

それでは、MongoDBが正しく立ち上がっていてかつPyMongoが正しくインストールされているのを確認しましょう。

In [ ]:
import pymongo
client = pymongo.MongoClient(host='localhost', port=27017)
client

上記セルで`MongoClient`というオブジェクトが返されたら、`pymongo`が正しくインストールされていることになります。`MongoClient`はPythonとMongoDBをつなぐオブジェクトで、MongoDBに対する操作はすべてこの`MongoClient`のメソッドを通して行います。例えば、今つながっているMongoDBにある全てのデータベースを表示するためには`database_name`というメッソドを使います。

In [ ]:
client.database_names()

上記コマンドを実行できてデータベース一覧が表示されていればMongoDBも問題なく立ち上がっているでしょう。ちなみに、データベース一覧には`admin`と`local`という2つのデータベースが含まれているのですが、これらはユーザ認証やレプリケーション等の高度な操作に使われています。しかし、本講座の範囲外の内容になるので、詳細な説明は割愛します。

### 10.2.4 PyMongoの基本操作

さて、準備ができたところで、実際に`MongoClient`オブジェクトを使ってMongoDBにデータにデータを書き込んだり、データを読み出したりしましょう。まず、使用するデータベースを選択する必要があります。今回はTwitterデータを扱うので、`twitter`というデータベースを選択しましょう。

In [ ]:
db = client.get_database('twitter')

厳密にいうと、この時点ではデータベースはまだ存在しないですが、最初のコレクションの最初のドキュメントを挿入するときにデータベースが自動的に作られます。コレクションもデータベースと同様に、事前に作る必要はありません。それでは、最初のドキュメントを`tweets`コレクションに入れましょう。

In [ ]:
result = db.tweets.insert_one({
    'created_at': 'Mon Jan 22 06:42:41 +0000 2018',
    'id': 955329854605537281,
    'lang': 'ja',
    'text': 'python使えるとWEBからデータ解析まで幅広いところで重宝されるから今のうち基礎固めとくといいことあるぜ'
})
result.acknowledged, result.inserted_id

これで、`db`というデータベースの`tweets`というコレクションにドキュメントを書き込んだことになります。書き込みを行うとき、PyMongoは書き込みの結果に関する情報を返します。それの中身をみると、書き込みが成功したかどうかや書き込まれたドキュメントの`ObjectID`等を確認できます。

また、上記の書き込みで、コレクションを`db`の属性として選択しているのですが、実はPyMongoではデータベースおよびコレクションは**属性**・**添字**・**メッソド**の３つの方法で選択できます。すなわち:

In [ ]:
all([client.foo.bar == client['foo']['bar'],
     client.foo.bar == client.get_database('foo').get_collection('bar'),
     client['foo']['bar'] == client.get_database('foo').get_collection('bar')])

>#### 使用するデータについて

>これから使うデータはすべてTwitterが提供している[Standard search API](https://developer.twitter.com/en/docs/tweets/search/api-reference/get-search-tweets)を使って取得したものです。取得したデータは「python」というキーワードの検索結果から作ったものです。

>本節ではMongoDBの基本操作の説明の観点から、APIから取得した生のデータではなく、一部のフィールドを省略したものを使うことにします。しかし、[10.3.5節](#10.3.5-データの集計)以降ではAPIから取得した生データを扱うことにします。

先程の書き込みではPythonの辞書(`dict`)を`insert_one`の引数として渡して、MongoDBにデータ挿入しています。MongoDBに書き込めるデータはミュタブルなマッピング型であればほぼなんでも挿入できます。MongoDBの内部では[BSON](http://bsonspec.org/)というJSONに似たようなデータ構造が使われていますが、`dict`からの変換はPyMongo等の各種ドライバがしてくれます。

次に、先程挿入したドキュメントを読み出すには、コレクションの`find`もしくは`find_one`メソッドを使います。

In [ ]:
db.tweets.find_one()

In [ ]:
list(db.tweets.find())

ここで、`find_one`の場合は結果がそのまま返されますが、`find`の場合は**カーソル**が返されます。ここで`list`を使っているのは、カーソルをイテレーションすることでデータを取得しているからです。カーソルの詳細については[10.3.2.1節]((#10.3.2.1-カーソル操作))で説明します。

MongoDBのコレクションに固定スキーマがないことの特徴を活かすことで、例えば以下のドキュメントも同じ`tweets`コレクションに挿入することができます。

In [ ]:
db.tweets.insert_one({
    'created_at': 'Mon Jan 22 09:11:33 +0000 2018',
    'id': 955367321220067333,
    'lang': 'ja',
    'text': '@petrol0110 @setuko1234 ワタシはプログラミング言語としてのPythonをちょっとかじった程度なので、統計ツールとしてのPythonは良く知らないんで何とも言えないんだけど。どんなもんかかじってみるのは損には… https://t.co/vFAbolzrrk',
    'entities': {
        'hashtags': [],
        'symbols': [],
        'user_mentions': [
            {'screen_name': 'petrol0110',
             'name': '石油王',
             'id': 838950516742770689, 
             'id_str': '838950516742770689',
             'indices': [0, 11]},
            {'screen_name': 'setuko1234',
             'name': 'せつこ山(のりこ山)後で一緒に風呂行く？',
             'id': 420145642,
             'id_str': '420145642',
             'indices': [12, 23]}],
        'urls': [
            {'url': 'https://t.co/vFAbolzrrk',
             'expanded_url': 'https://twitter.com/i/web/status/955367321220067333',
             'display_url': 'twitter.com/i/web/status/9…',
             'indices': [117, 140]}
        ]
    }
})

最初に入れたドキュメントには`entities`というフィールドがなかったのですが、MongoDBではあとから入れるドキュメントに新しいフィールドがあったり、前に挿入したドキュメントにあったフィールドがなかったりしても全く問題ありません。

ここで、もう一度`find`を実行すると、違うスキーマを持つ2つのドキュメントが同じコレクションに入っていることを確認できます。

In [ ]:
list(db.tweets.find())

### 10.2.5 MongoDBのデータ型

前節では複数のフィールドを持つ2つのツイートを挿入しましたが、それぞれのフィールドの各値は特定の型として認識されデータベースに格納されています。RDBMSと同様に、MongoDB/BSONにはいくつかのデータ型が用意されています。先程みたように、ドキュメント挿入時に明示的に型を定義する必要はありませんが、PyMongo等のドライバが自動的にデータを適当な型に変換してデータベースに格納します。当然ですが、ドライバが解釈できないデータを挿入しようとしたら何らかのエラーが発生します。

ここで、最も使われるデータ型<sup id="a2">[2](#f2)</sup>を紹介します

<p id="table1" style="text-align:center">**表1**</p>

| 型の種類 | PyMongo使用時の具体例 |
| :-----: |:-----------|
| 浮動小数点数     | `{"x": 2.71}` | 
| 文字列   | `{"x": 'foo'}` | 
| 日時     |`{"x": datetime.datetime.now()}` | 
| ブーリアン | `{"x" : True}`|
| 配列      |`{"x": ["a", 2, "c"]}` |
| 埋め込みドキュメント | `{"x" : {"foo" : "bar"}}` | 
| Object ID | `{"x": ObjectId("589810d9c75c4997748e77ef")}`



<sup id="f2">**[2]** 上記の表のもの以外に、正規表現型やバイナリ型など、MongoDBには様々な型があります。詳しくは[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/bson-types/)を見てください。[↩](#a2)</sup>

数字用の型はいくつかあるのですが、Pythonの`int`や`float`はそれぞれ適当な浮動種数点数や整数用の型として扱われます。

文字列は任意のUTF-8で符号化されたものを扱うことができます。すなわち、Python 3の`str`型ならなんでもそのまま挿入できます。なお、MongoDB/BSONではUTF-8以外の文字符号化方式を使うことが不可能です。

日時はUNIXエポックからのミリ秒の数を表す整数として保存されます。また、時間帯サポートがないので、すべての日付はUTCとして保存されます。マイクロ秒・ナノ秒単位の時間を保存する必要がある場合は整数用の違う型を使うことが必要です。

ブーリアン型は真(`True`)か偽(`False`)を保存します。

配列では任意の型の複数の値のものを保存するのに使います。MongoDBの大きな特徴の一つはやはり配列を[第一級オブジェクト](https://ja.wikipedia.org/wiki/第一級オブジェクト)(*first-class object*)として扱っていることです。これは非常に便利でRDBMSにはない特徴です。また、MongoDBの配列はPythonのリスト(`list`)と同様に異なる型の値を同じ配列に納めることが可能です（上の表の例を参照）。

埋め込みドキュメント(*embedded document*)とは他のドキュメントのキーの値として使われるドキュメントのことです。埋め込みドキュメントを使うことでRDBMSのようにデータ構造をフラットなものに限定する必要がありません。先程データベースに挿入した2つ目のツイートの`entities`フィールドは実は一つの要素の配列の中に一つの埋め込みドキュメントが入っていました。このように、配列や埋込ドキュメントを使うことで、複雑な階層構造のデータをそのまま保存することができます。

#### `ObjectID`/`_id`フィールドについて

前節で示した[`find`結果](#10.2.4-find)には`_id`フィールドが追加されているのを不思議に思った読者も多いでしょう。実は、MongoDBでは全てのドキュメントには必ずユニークな`_id`フィールドが与えられます。一般的に、MongoDBが自動的にその`ObjectId`を生成しますが、ユーザが明示的に指定することもできます。

`ObjectId`の生成方法は複数のマシン・プロセスが同時に大量のドキュメントを挿入しても問題が起こらないように設計されています。`ObjectId`の最初の4バイトはUNIX時間、次の3バイトはマシンID、次の2バイトはプロセスID、そして最後の3バイトはランダムに初期化されるカウンタです（下記図を参照）。もし、より単純な生成方法を使っていれば、多くのクライアントから同時にアクセスがあったときに、重複`ObjectId`が発生しエラーが発生しかねないため、このような設計になっています。また、詳しくは[10.4.1節](#10.4.1-インデックス)で述べますが、`ObjectId`はすべてのコレクションで自動的に作成されるデフォルトインデックスにも使われます。

![img](https://docs.google.com/drawings/d/1HBZMNvd8WsnoUKzImPRkpIoKwx-rHOp5AuyomIjA2PQ/pub?w=1200)

***

## 10.3 MongoDBの基本操作

本節ではTwitterデータを使って、MongoDBのいわゆるCRUD操作(create, read, update, delete)および集計操作について学びます。それらの操作はすべて**コレクションに対して**行うものです。各操作に対する代表的なメソッドの概要を以下の表にまとめています。

<p id="table2" style="text-align:center">**表2**</p>

| 操作 | 代表的なメソッド |
| :-----: |:-----------|
| Create  | `db.collection.insert_one(<ドキュメント>, [<オプション>, ...])` <br> `db.collection.insert_many(<ドキュメントの配列>, [<オプション>, ...])`| 
| Read    | `db.collection.find_one(<フィルタ>, <プロジェクション>, [<オプション>, ...])`  <br> `db.collection.find(<フィルタ>, <プロジェクション>, [<オプション>, ...])`| 
| Update  | `db.collection.update_one(<フィルタ>, <アップデート>, [<オプション>, ...])` <br> `db.collection.update_many(<フィルタ>, <アップデート>, [<オプション>, ...])` <br> `db.collection.replace_one(<フィルタ>, <新しいドキュメント>, [<オプション>, ...])`| 
| Delete  | `db.collection.delete_one(<フィルタ>, [<オプション>, ...])` <br> `db.collection.delete_many(<フィルタ>, [<オプション>, ...])` <br> `db.collection.drop()`| 

本節ではそれらを順番に詳しく見ていきます。この表2は何度か言及するので、しっかり覚えておくようにしましょう。また、必要に応じて、[PyMongoの公式ドキュメンテーション](https://api.mongodb.com/python/current/api/pymongo/collection.html)も参考にしてください。

### 10.3.1 データの挿入 (Create)



[10.2.4節](#10.2.4-Mongoシェルの基本操作)では下記のように`insert`を使って**1つ**のドキュメントを挿入しました。

In [ ]:
db.tweets.insert_one({
    'created_at': 'Mon Jan 22 09:02:39 +0000 2018',
    'id': 955365080035667968,
    'lang': 'fr',
    'text': 'RT @LaTourDuWeb: Vendredi 19 janvier @LaTourDuWeb accueille une nouvelle session @LyonDataScience qui portera sur la Place de R et Python d…'
})

それ以外に、複数のドキュメントを入れるときはドキュメントを配列にまとめて、`insert_many`を使います。マッピング型を要素とする`list`などのイテレーション可能(*iterable*)なものなら引数はなんでも大丈夫です。

In [ ]:
db.tweets.insert_many([
    {'created_at': 'Mon Jan 22 09:24:34 +0000 2018',
     'id': 955370596229148673, 
     'lang': 'en',
     'text': 'RT @mikedailly: Who knew there were two different single markets? Labour’s position on #Brexit is the dead parrot in Monty Python’s sketch.…'
    },
    {'created_at': 'Mon Jan 22 08:39:56 +0000 2018',
     'id': 955359364310646784,
     'lang': 'ja',
     'text': '@sylvan5 データの読み込みは TensorFlow のモジュールを使って、モデルの部分は tf.keras を使うという手が。。。https://t.co/hueLxCqEF3',
    },
])

ここで、これ以降使うデータセートをファイルから読み込み、まとめてMongoDBに挿入します。しかし、その前に、これまでで入れたデータを削除するために、まずは`tweets`コレクションをリセットしましょう。

In [ ]:
db.tweets.drop()

次に、圧縮JSONファイルを読み込み、`insert_many`でMongoDBに挿入しましょう。（なお、python_tweets.json.gzというファイルを使いますので、そのファイルがあるパスを以下で指定して、読み込んでください。）

In [ ]:
from bson import json_util
import gzip

with gzip.open("mongo_data/python_tweets.json.gz") as file:
    data = [json_util.loads(line) for line in file]
    db.tweets.insert_many(data)

>**注意**: PyMongoには`insert_one`や`insert_many`以外に`insert`というメソッドもありますが、現在のMongoDB v3.4では、多くの言語のドライバでは**非推奨**(deprecated)扱いになっているので、使わない方が無難でしょう。

### 10.3.2 データの検索 (Read)


データを検索するのに`find`メソッドを使いますが、一般的に対象データを指定するために**クエリフィルタ**というパラメータを渡します。**クエリフィルタ**はSQL構文の`where`によく似ており、特定の条件にマッチしたドキュメントのみを取得するのに使います。

最も単純なクエリドキュメントはコレクションの全てのドキュメントにマッチする空フィルタ`{}`です。[10.2.4節](#10.2.4-Mongoシェルの基本操作)では、`find`に何も引数を渡さなかったのですが、それは`{}`を渡すのと同じです。あるフィールドが特定の値になっているドキュメントのみを検索した場合は`{<field>: <value>}`のようなフィルタを渡せばいいです。例えば、日本語のツイートを一つだけ検索したい場合は、`find_one`に`{'lang': 'ja'}`を使えばいいです。

In [ ]:
db.tweets.find_one({'lang': 'ja'})

#### プロジェクション

[表2](#table2)にあるように、`find`/`find_one`には**プロジェクション**というもう一つのパラメータを渡せます。プロジェクションはSQLでいう`SELECT`構文のようなもので、検索にマッチしたドキュメントの取得するフィールドを指定するのに使います。
例えば、ツイートの本文**だけ**を取得したい場合は次のようにしてします。

In [ ]:
db.tweets.find_one({}, {'text': 1})

デフォルトで`_id`も返されますが、プロジェクションを下記のようにすると`_id`フィールドを省略できます。

In [ ]:
cursor = db.tweets.find({}, {'text': 1, '_id': 0})
for _ in range(5):
    print(next(cursor))

基本的に一つのプロジェクションではフィールドの明示的な選択(`1`)と明示的な排除(`0`)を混ぜることができません（唯一の例外は先程みた`_id`フィールドです）。少し考えればその理由はわかると思いますが、とにかくどちらかを選択する必要があります。したがって、例えば、次のクエリでは`OperationFailure`というエラーが発生します。

In [ ]:
try:
    list(db.tweets.find({}, {'created_at': 0, 'text': 1}))
except pymongo.errors.OperationFailure as e:
    print(repr(e))

しかし、明示的なフィールド選択**だけ**なら可能です。

In [ ]:
list(db.tweets.find({}, {'created_at': 1, 'text': 1}))[-5:]

逆に、明示的なフィールドの排除**だけ**なら可能です。

In [ ]:
db.tweets.find_one({}, {'_id': 0, 'entities': 0, 'user': 0})

#### クエリ演算子

MongoDBの大きな特徴としては「`$`」で始まる**演算子**(operator)です。演算子は様々な用途に使われますが、クエリでよく使う演算子には以下のものがあります。

<p id="table3" style="text-align:center">**表3**</p>

| 演算子 | 説明 |
| :-----: |:-----------|
| `$eq`   | equal, 等しい|
| `$lt`   | less than, 未満|
| `$lte`  | less than or equal, 以下     |
| `$gt`   | greater than, より大きい    |
| `$gte`  | greater than or equal, 以上     |
| `$exists`  | フィールドが存在するかどうか     |
| `$in`  | 与えられた値が配列に含まれているかどうか     |
| `$or`  | 複数のクエリの論理和     |
| `$and`  | 複数のクエリの論理積     |
| `$not`  | 論理積否定     |
| `$regex`  | 正規表現     |


実は先程紹介した`{<field>: <value>}`は`{<field>: {$eq: <value>}}`を意味する[シンタックスシュガー](https://ja.wikipedia.org/wiki/糖衣構文)でした。

例えば、言語が英語で、ツイートの`id`が`955366928331300864`より大きいツイートが欲しい場合は上記の`$gt`演算子を使って、下記のように検索できます（プロジェクションも同時に渡しています）。

In [ ]:
list(db.tweets.find({'lang': 'en', 'id': {'$gt': 955366928331300864}},
                    {'_id': 0, 'text': 1, 'id': 1}))[:5]

クエリでもプロジェクションでもそうですが、複数のフィールドを指定することで、それらの論理積（[`AND`](https://ja.wikipedia.org/wiki/論理積)）をとって検索することになります。例えば、先程の検索で使ったクエリドキュメントの場合は`lang`が`en`**かつ**`id`が`955366928331300864`より大きいもののみが返されます。

他の演算子を使うと、様々な検索ができます。例えば、`$exists`を使うことでフィールドの有無を検索条件にすることもできます。

In [ ]:
list(db.tweets.find({'retweeted_status': {'$exists': True}}, {'_id': 0, 'text': 1}))[:5]

`$in`は特定の値が配列の中に含まれているもののみを検索するのに使います。`$in`で使う配列の要素が複数の場合は論理和([`OR`](https://ja.wikipedia.org/wiki/論理和))をとって検索します。

In [ ]:
list(db.tweets.find({'entities.hashtags.text': {'$in': ['pandas', 'flask']}},
                    {'_id': 0, 'text': 1}))

上記の検索では、「pandas」もしくは「flask」のハッシュタグが付いているツイートが返されます。ここで、クエリに`entities.hashtags.text`を使うことで、**埋め込みドキュメント**のフィールドや**配列**の中身を直接検索することができてしまいます。この機能は便利ですが、その機能があることでフィールド名に「`.`」を使うことが禁止されています。


なお、`$in`の対象フィールド（上記の例で`entities`)の値は配列である必要がないので、以下のような検索も有効です。

In [ ]:
# タイ語とベトナム語のツイート検索
list(db.tweets.find({'lang': {'$in': ['th', 'vi']}},  {'_id': 0, 'text': 1, 'lang': 1}))

また、文字列の検索<sup id="a2">[2](#f2)</sup>をしたい場合は、正規表現演算子の`$regex`を使うと便利です。

<sup>**[2]** テキストインデックスを利用することで`$text`演算子でより高度な文字列検索が可能です。詳しくは[10.4.1節](#10.4.1-インデックス)および[公式ドキュメンテーション](https://docs.mongodb.com/manual/text-search/)を見てください[↩](#a2)</sup>

In [ ]:
import re
regex = re.compile('mongo', re.IGNORECASE)
list(db.tweets.find({'text': {'$regex': regex}},  {'_id': 0, 'text': 1}))

こちらでの説明は省略しますが、正規表現にについてもっと知りたい読者は[Wikipedia](https://ja.wikipedia.org/wiki/正規表現)や[`re`モジュール](https://docs.python.org/3/library/re.html)を参考にしてください。

ここで紹介したのはよく使う基礎的な演算子ですが、MongoDBには他にたくさんの演算子があります。クエリ用演算子の詳細については[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/query/#query-selectors)を見てみてください。

**<練習問題1>**

ハッシュタグがちょうど7つあるツイートを数を求めよ。(ヒント: `$size`演算子を使う)

In [ ]:
## ここでコードを書いたり、セ`ルを追加したりしてください

#### 10.3.2.1 カーソル操作

[10.2.2節](#10.2.2-MongoDBの仕組み)で述べたように、MongoDBに対して検索を行うときに返されるのは**カーソル**です。`find`はカーソルを返しますが、そのカーソルの実際の実行・データ取得・表示は必要になるまで行われません<sup id="a3">[3](#f3)</sup>。これまで見てきたようにPyMongoではカーソルに対して何らかの操作(例えば、`list()`をコールする等）をしないと、実際のデータ取得・表示は行われません。

<sup>**[3]** これを[遅延評価](https://ja.wikipedia.org/wiki/遅延評価)(lazy evaluation)といい、関数型言語でよく使用されるテクニックです。[↩](#a3)</sup>

これまでは**コレクションに対して**行う操作を紹介してきましたが、ここでは**カーソルに対して**行ういくつかの便利な操作を以下に紹介します。必要に応じて、[PyMongoの公式ドキュメンテーション](https://api.mongodb.com/python/current/api/pymongo/cursor.html)も参考にしてください。

<p id="table4" style="text-align:center">**表4**</p>

| メソッド | 説明 |
|:-----------|:-----------|
| `cursor.sort(<ソート>)` | 検索結果を整列する  |
| `cursor.count()` | 検索結果のドキュメントの数を返す　|
| `cursor.limit(<数>)` | 返されるドキュメント数を制限する |
| `cursor.skip(<数>)` | 返されるを<数>だけスキップする |

例えば、各ツイートの言語だけに注目すると、

In [ ]:
list(db.tweets.find({}, {'_id': 0, 'lang': 1}))[:10]

整列されていないことがわかります。ここで、ソートドキュメントを使って以下のように整列できます。なお、スペース節約のため最初の10ツイートだけ表示していますが、毎回データベースにある全ツイートを取得しています。

In [ ]:
list(db.tweets.find({}, {'_id': 0, 'lang': 1}).sort('lang'))[:10]

ソートドキュメントにおいて、`1`は昇順、`-1`は降順を意味します。複数のフィールドを組み合わせることも可能です。

In [ ]:
cursor = db.tweets.find({}, {'_id': 0, 'id': 1, 'entities.hashtags.text': 1})
list(cursor.sort([('entities.hashtags.text', -1), ('id', 1)]))[:10]

また、PyMongoでは、便宜的に`1`には`pymongo.ASCENDING`、 `-1`には`pymongo.DESCENDING`というaliasが定義されていますので、上記のセルは下記のように書き換えることも可能です。

In [ ]:
cursor = db.tweets.find({}, {'_id': 0, 'id': 1, 'entities.hashtags.text': 1})
list(cursor.sort([('entities.hashtags.text', pymongo.DESCENDING),
                  ('id', pymongo.ASCENDING)]))[:10]

上記の検索では`hashtags`に対して降順で整列したのち、さらに`id`で昇順で整列しています。

また、上記の検索で、取得されるドキュメント数を制限したい場合は`limit`を使います。

In [ ]:
list(db.tweets.find({}, {'_id': 0, 'id': 1, 'entities.hashtags.text': 1})
              .sort([('entities.hashtags.text', -1), ('id', 1)])
              .limit(5))

また、最初のいくつかのドキュメントをスキップしたい場合は`skip`を使えばいいです。

In [ ]:
list(db.tweets.find({}, {'_id': 0, 'id': 1, 'entities.hashtags.text': 1})
              .sort([('entities.hashtags.text', -1), ('id', 1)])
              .skip(5))[:5]

検索結果のドキュメントの個数だけを知りたい場合は`count`を使います。

In [ ]:
db.tweets.find({'lang': {'$in': ['es', 'ja']}}).count()

また、コレクション全体のドキュメント数を知りたい場合は下記のようにできます。

In [ ]:
db.tweets.find().count()

なお、`db.tweets.count()`でも同じ結果が得られます。これもシンタックスシュガーです。

`sort`のようにカーソルに対して行う操作がまたカーソルを返す場合、上記の例のように複数の操作を繋ぎ合わせることができます。しかし、当然ながら`count`のようにカーソルではなく、数値等を返す操作の場合はできません。

ここで見てきたカーソル操作はすべてPython側で行うことが可能なので、一見不要なコマンドに思う読者もいるかもしれません。しかし、ここで紹介したメソッドを使うことで**MongoDB側**で操作を行い、必要なデータだけをPythonに送るのに対し、Pythonで全部やろうとすると、一旦すべてのデータを読み込んでから処理することになります。MongoDBサーバとPythonクライアントが違うマシンにあるときは無駄なデータ通信が発生します。また、通信問題がなくても、検索結果をすべてメモリ上に載せる必要があるのですが、データ量が膨大な場合はエラーが発生しかねません。通信・メモリの問題がなくても、ほとんどの場合にはMongoDB側で処理した方が圧倒的に高速です。例えば、下記の2つのセルではそれぞれMongoDB側とPython側で全く同じ処理をしたときの実行時間を比較します。

In [ ]:
%%time
result = list(db.tweets.find().skip(5).limit(5))
print(len(result))

In [ ]:
%%time
result = list(db.tweets.find())[5:10]
print(len(result))

筆者が執筆時に調べたとき、今回のデータセットでは約300倍のスピードの違いが出ました。データが多ければ多いほどその違いが大きくなるでしょうから、MongoDB側でできる処理はなるべくMongoDBにやらせるようにした方がよいでしょう。

しかし、ただMongoDB側で操作をすれば速くなるとは限りません。特に、ソートをするときには通常、**インデックス**が使われます。今回はインデックスを使わなかったのですが、大きなデータセットをインデックスなしでソートしようとしたら、非常に時間がかかるかエラーが発生します。[10.4.1節](#10.4.1-インデックス)で詳しく述べますが、検索やソートでよく使うフィールドについてはインデックスを作る必要があることを覚えておいてください。

### 10.3.3 データの更新 (Update)

データの更新を行うとき、1)あるドキュメントを丸ごと他のドキュメントと入れ替える、2)一つもしくは複数のドキュメントの特定のフィールドに何らかの操作をする、の2つのパターンがあります。1)では`replace_one`を使い、2)では`update_one`もしくは`update_many`を使います。各メソッドのシグネチャは[表2](#table2)を参照してください。

#### ドキュメントの入れ替え

今回のTwitterデータセットにはタイ語のツイートが一つだけあります。

In [ ]:
list(db.tweets.find({'lang': 'th'}, {'text': 1}))

あまり現実的な例ではないですが、仮にそのタイ語ツイートを`{'foo': 'bar'}`というドキュメントと入れ替えたい場合、下記のコマンドを実行すればいいです。

In [ ]:
db.tweets.replace_one({'lang': 'th'}, {'foo': 'bar'})

先程の`find`コマンドをもう一回実行すると、タイ語のツイートがなくなっていることが確認できます。一方で、次のコマンドで、新しく先程の`{'foo': 'bar'}`ドキュメントが挿入されたことがわかります。また、`_id`を上記の`find`結果と比較すると、同じドキュメントであることがわかります。

In [ ]:
list(db.tweets.find({'foo': {'$exists': True}}))

このような操作はそもそもなぜ必要なのかと思われるかもしれませんが、実は各ドキュメントに対してMongoDBだけでは対処しきれない操作をしたい場面では便利です。例えば、ツイートの本文をGoogle翻訳APIを使って翻訳したい場合はPython等からMongoDBからデータを取得し、翻訳やその他の操作をしてから元のドキュメントと入れ替えたいときには、`replace_one`を使うと削除操作と挿入操作を同時にできてしまうので一石二鳥です。

#### フィールド操作と更新演算子

一般的に、ドキュメントを丸ごと入れ替えるより、ドキュメントの特定のフィールドだけを変更することが多いでしょう。Twitterの場合はツイートがリツイートされたら、そのリツイートカウントに関するフィールドだけを変更したい例がわかりやすいでしょう。

MongoDBでは、こういった更新操作のための演算子をいくつか用意しています。本節では下記の**表5**にある3つだけを紹介しますが、他にもたくさんあるのでぜひ[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/update/)を見てみてください。

<p id="table5" style="text-align:center">**表5**</p>

| 演算子 | 説明 |
| :-----: |:-----------|
| `$set`   | フィールドの値を設定 |
| `$inc`   | 数値を増減 |
| `$push`  | 配列に値を挿入 |


- **`$set`**

下記のツイートは中身が英語にも関わらず、言語が未定(`und`)になっています。

In [ ]:
from bson import ObjectId

query_filter1 = {'_id': ObjectId('5a65aa832b2a8000afa26884')}
db.tweets.find_one(query_filter1, {'text': 1, 'lang': 1})

もし、このツイートの言語フィールドを英語に変更したい場合は`$set`を下記のように使えばいいです。

In [ ]:
result = db.tweets.update_one(query_filter1, {'$set': {'lang': 'en'}})
result.raw_result

もう一回`find`を使うと、確かに言語の値が変更されていることがわかります。また、当然`_id`が変わっていないので、前と同じドキュメントであることも確認できます。

In [ ]:
db.tweets.find_one(query_filter1, {'text': 1, 'lang': 1})

今回は一つだけのフィールドを変更しましたが、複数のフィールドを同時に変更することも可能です。

- **`$inc`**

先程のリツイートのカウントの変更について述べましたが、具体的には下記のツイートの例を考えましょう。

In [ ]:
query_filter2 = {'_id': ObjectId('5a65aa832b2a8000afa26865')}
db.tweets.find_one(query_filter2, {'text': 1, 'retweet_count': 1, 'entities.hashtags.text': 1})

仮にこのツイートがリツイートされて、`retweet_count`に1を足したい場合は以下のコマンドを実行すればいいです。

In [ ]:
result = db.tweets.update_one(query_filter2, {'$inc' : {'retweet_count': 1}})
result.raw_result

先程の`find`コマンドをもう一回実行すると`retweet_count`が増えていることがわかります。

このような場合は`find`で現在の`retweet_count`を取得して、`$set`でそれを変更することも可能ですが、`$inc`の方が一回の操作で済みます。また、`update_many`で複数のドキュメントを同時に更新するときは`find`+`$set`より便利であることが想像できるかと思います。

- **`$push`**

Twitter上でツイート本文を変更することはできないですが、仮にできたとして、先程のツイートの本文の「Python」という単語が「#Python」というハッシュタグに変更されたとしましょう。その場合は`hashtags`フィールドの配列に`{'text': 'Python'}`を追加することになります(他のフィールドについては省略します）が、そのような操作は下記のように`$push`演算子を使います。

In [ ]:
result = db.tweets.update_one(query_filter2, {'$push': {'entities.hashtags': {'text': 'Python'}}})
result.raw_result

先程の`find`コマンドをもう一回実行すると`hashtags`の配列に新しい要素が追加されたことがわかります。

#### Upsert

`update_one`と`update_many`のには何種類かのオプションパラメータを([表2](#table2)を参照）渡せますが、最も使うオプションの一つは`upsert`というものです。Upsertとは「update」と「insert」の造語でそれを使うと、検索クエリにマッチしたドキュメントがあればそのドキュメントを更新し、無ければドキュメントを挿入します。

これまでツイッターデータを使っていましたが、今回はある会社の社員名簿を扱っているとしましょう。そして、「Taro Mongo」という社員のメールアドレスを設定するため、以下のコードを考えます。

In [ ]:
result = db.contacts.update_one(
    {'name': 'Taro Mongo'},
    {'$set': {'email': 'taro.mongo@mongodb.com'}}
)
result.raw_result

結果を見ると、フィルタ`{'name': 'Taro Mongo'}`にマッチしたドキュメントがなかったため、更新(`nModified`)が行われなかった、かつ、挿入されたドキュメント`n`がゼロであったことがわかります。とろこが、`{'upsert': True}`を使うと、

In [ ]:
result = db.contacts.update_one({'name': 'Taro Mongo'},
                           {'$set': {'email': 'taro.mongo@mongodb.com'}},
                           upsert=True)
result.raw_result

ドキュメントが挿入されます。それを確認すると、フィルタとアップデートドキュメントを合成したようなドキュメントになっていることが分かります。

In [ ]:
db.contacts.find_one()

#### 複数のドキュメントを扱う場合

これまでは`update_one`だけを使って、常に一つだけのドキュメントを更新する例を示してきましたが、`update_many`は`update_one`とほぼ全く同じように使います。例えば、Twitterデータセットにすべてのツイートに既読(`read`)フィールドを追加したい場合は以下のようにすればいいです。

In [ ]:
result = db.tweets.update_many({}, {'$set': {'read': True}})
result.raw_result

>**注意**: PyMongoには`update_one`や`update_many`以外に`update`というメソッドもありますが、現在のMongoDB v3.4では、多くの言語のドライバでは**非推奨**(deprecated)扱いになっていますので、`insert`と同様に使わない方が無難でしょう。

### 10.3.4 データの削除 (Delete)

CRUD操作の最後の「delete」ですが、CRUDの4操作の中最も単純なものです。例えば、全ての日本語のツイートを削除コマンドを実行すると、

In [ ]:
result = db.tweets.delete_many({'lang': 'ja'})
result.raw_result

念のため、日本語ツイートの数を確認すると、

In [ ]:
db.tweets.find({'lang': 'ja'}).count()

空フィルタで`delete_many`を実行すれば、そのコレクションのすべてのドキュメントが削除されますが、単にコレクションを丸ごと削除したい場合は実は`drop`というメソッドの方が早いです。[表2](#table2)にあるように、`drop`はパラメータを取らないので、今までのツイートをすべて消すには以下のコマンドを実行します。

In [ ]:
db.tweets.drop()

また、`drop`を使う場合、ドキュメントだけではなく、コレクションに付随されるインデックス等のメタデータもすべて削除されます。

>**注意**: PyMongoには`delete_one`や`delete_many`以外に`remove`というメソッドもありますが、現在のMongoDB v3.4では、多くの言語のドライバでは**非推奨**(deprecated)扱いになっていますので、`insert`と同様に使わない方が無難でしょう。

### 10.3.5 データの集計

MongoDBで集計処理を行う方法は3つあります。

1. **単一目的操作**:
これまでみてきたような`db.collection.count()`等の一つのコレクションのみを操作対象とする簡易集計です。便利な方法ですが、その反面、高度な集計操作には向いていません。
1. **Aggregationフレームワーク**:
UNIXと似たようなパイプラインの概念を用いて、ドキュメントに対して順番にいくつかの操作を行い、最終的に集計ドキュメントを出力します。SQLでいうGROUP BYやWHERE構文に相当します。検索と更新と同じように、特殊な演算子を使って操作します。
1. **MapReduce機能**:
Map関数およびReduce関数を独自に定義し，集計処理を行う方法です。集計パイプラインではできないような複雑な集計処理を行うために使用されます。直接Javascriptの関数を書く必要があるのと、上級者向けであることから、説明は省略します。

本節では集計パイプラインを中心に紹介します。

#### データベース再構築

[10.3.4節](#10.3.4-データの削除-&#40;Delete&#41;)でTwitterデータセットのデータをすべて削除してしまいましたので、[10.3.1節](#10.3.1-データの挿入-&#40;Create&#41;)でやったようにツイートをまた挿入しましょう。（ここでもpython_tweets.json.gzというファイルを使いますので、そのファイルがあるパスを以下で指定して、読み込んでください。）

In [ ]:
from bson import json_util
import gzip

with gzip.open("mongo_data/python_tweets.json.gz") as file:
    data = [json_util.loads(line) for line in file]
    db.tweets.insert_many(data)

#### 集計パイプラインの概要

Aggregationフレームワークでは、コレクション内のドキュメントを変換および結合を繰り返すことで、欲しい集計データを取得します。 フィルタリング、プロジェクション、グループ化、並べ替え、リミット、スキップ等の変換・結合操作をパイプラインのようにつないでいくことで処理を行います。後ほど紹介しますが、これらの変換・結合操作を行うには**Aggregationフレームワーク演算子**を使います。

たとえば、Twitterデータセットで最も多く使われた言語を知りたい場合、以下のパイプラインを考えます。
1. 各ツイートの言語を取得
1. ツイートを言語毎にグループ化して各言語のツイートの数を数える
1. ツイートが多い言語の順に並び替える
1. 上位5言語のみを表示

上記の各操作は集計パイプラインにおける一つの**ステージ**に相当します。各ステージでデータに何らかの変換・結合操作を行い、その結果を次のステージの入力として渡します。各ステージの入力ドキュメント数と出力ドキュメント数は必ずしも一致する必要がないです。一般的にドキュメント数を減らしていきますが、ドキュメント数が増えるステージもありえます。

各ステージは一つだけの**ステージ演算子**(*stage operator*)をフィールドに持ち、そのフィールドの値に**パイプライン表現**と呼ばれる特殊なドキュメントを持ちます。パイプライン表現ではこれまでみてきたようなクエリドキュメントやプロジェクションドキュメントに似たようなドキュメントが使われます。

それでは、上記の4ステップの操作をAggregationフレームワークで書くと以下のようになります。

In [ ]:
cursor = db.tweets.aggregate([
    {"$project": {"lang": 1}},
    {"$group": {"_id": "$lang", "count" : {"$sum" : 1}}},
    {"$sort": {"count": -1}},
    {"$limit": 5}
])
list(cursor)

これで最も多い言語は英語であることがわかります。

また、例えば、日本語のツイートの中でハッシュタグの数毎にツイートをグループ化して、グループ毎の合計ツイート数を求めたい場合は以下のようになります。

In [ ]:
cursor = db.tweets.aggregate([
    {"$match": {"lang": "ja"}},
    {"$project": {"num_hashtags": {"$size": "$entities.hashtags"}}},
    {"$group": {"_id": "$num_hashtags", "count" : {"$sum" : 1}}},
    {"$sort": {"_id": 1}}
])
list(cursor)

日本語のツイートの大半にはハッシュタグが付いていないことがわかります。

上記の2つの例でいくつかの新しい演算子が登場したので、順番に説明していきます。

- **`$project`**<br>
データ検索におけるプロジェクションドキュメントに似ていますが、検索のように単純にフィールドを選ぶだけではなく、フィールドを改名したり、**表現演算子**を使っていくつかの操作を加えたりできます。`$<フィールド名`>という表記を使うことで、入力ドキュメントにおけるそのフィールドの値に対して表現演算子の操作を行うことができます。<br>
具体的には上記の1つ目の例では、単純なフィールド選択しかしていませんが、2つ目の例では、`{$project: {"num_hashtags": {$size: "$hashtags"}}}`で`num_hashtags`という新しいフィールドを作り、その中身を`$hashtags`の配列の長さにしています。他に様々な数字操作や文字列操作が可能です（[⇗公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/aggregation/#expression-operators))。
<br><br>
- **`$group`**<br>
SQLでいう`GROUP BY`に非常に似ており、`_id`フィールドで指定した（一つもしくは複数の）フィールドに対してグループ化を行った上で、定義する他の新フィールドで**アキュムレータ**と呼ばれる種類の演算子を使って、集計操作を行います。上記の2つの例では、`count`という新フィールドを定義し、（ツイート毎に1を足すことで）ツイートの合計数を求めています。他に下記の例で示す配列に操作を行ったり、最大値・最小値等を求めたりなど、様々な操作が可能です。
<br><br>
- **`$sort`**<br>
[10.3.2.1節](#10.3.2.1-カーソル操作)でみた`sort`カーソルメソッドと同じ操作です。`1`で昇順、`-1`で降順にドキュメントを整列します。
<br><br>
- **`$limit`**<br>
[10.3.2.1節](#10.3.2.1-カーソル操作)でみた`limit`カーソルメソッドと同じ操作です。次のステージの渡すドキュメント数を指定した数だけに制限します。
<br><br>
- **`$match`**<br>
データ検索におけるクエリドキュメントとほぼ同じで、クエリで使える演算子もそのまま使えます。上記の2つめの例で、次のステージに渡すドキュメントを言語が英語であるものだけに制限するのに使っています。集計パイプラインにおいて、次のステージで処理するドキュメント数を減らすために、できるだけ早い段階で`$match`を使うことがよいとされています。
<br><br>
- **`$size`**<br>
配列の要素の数を返します。
<br><br>
- **`$sum`**<br>
数値の和を返します。非数値の値は無視されます。

上記の2つの例では最終的に数値を求めましたが、Aggregationフレームワークは他に様々な用途に使えます。例えば、ハッシュタグを言語毎にグループ化するのは次のように行います。

In [ ]:
cursor = db.tweets.aggregate([
    {"$project": {"hashtag": "$entities.hashtags.text", "lang": 1}},
    {"$unwind": "$hashtag"},                    
    {"$group": {"_id": "$lang", "hashtags": {"$addToSet": '$hashtag'}}},
])
list(cursor)

上記の例で新しくが登場した演算子を説明します。

- **`$unwind`**<br>
配列の各ドキュメントを別々のドキュメントに変換します。このステージ操作では一般的に出力ドキュメント数は入力ドキュメント数より多くなる傾向があります。上記の例では、各ツイートの各ハッシュタグを新しいドキュメントに分けるのに使っています。
<br><br>
- **`$addToSet`**<br>
グループ毎に指定したフィールドの値を集合配列にまとめます。数学的な意味での集合なので、重複値は無視され、出力配列における要素の順番をコントロールできます。

Aggregationフレームワークを使うにあたって、パイプラインでエラーが出たら、ステージを一つずつ試していくことでデバッグできますので、下記の練習問題で困ったらそのテクニックを利用してみてください。

集計用演算子は他にたくさんあるので、必要に応じて[公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/operator/aggregation/)を参考にしてください。これまで紹介したAggregationフレームワーク演算子を以下の表にまとめました。

<p id="table6" style="text-align:center">**表6: 本節で紹介したAggregationフレームワーク演算子**</p>

| 演算子 | 種類 |
| :-----:|:-----------:|
|`$project`|ステージ演算子|
|`$group`|ステージ演算子|
|`$sort`|ステージ演算子|
|`$limit`|ステージ演算子|
|`$match`|ステージ演算子|
|`$size`|表現演算子|
|`$sum`|アキュムレータ|
|`$unwind`|ステージ演算子|
|`$addToSet`|アキュムレータ|


**<練習問題2>**

データセットに登場するすべてのハッシュタグをunwindし、数の多いものの順位に上位20ハッシュタグを求めよ。

In [ ]:
##ここでコードを書いたり、セルを追加したりしてください

参考程度に、以下の対照表を載せておきます。


<p id="table7" style="text-align:center">**表7: SQL構文/MongoDB演算子の関係**</p>

| SQL構文 | MongoDB演算子 |
| :----- |:-----------|
|`WHERE`	|`$match`|
|`GROUP BY`	|`$group`|
|`HAVING`	|`$match`|
|`SELECT`	|`$project`|
|`ORDER BY`	|`$sort`|
|`LIMIT`	|`$limit`|
|`SUM()`	|`$sum`|
|`COUNT()`	|`$sum`|
|`join`		|`$lookup`|

***

## 10.4 MongoDBのパフォーマンス向上

### 10.4.1 インデックス

インデックスはMongoDBの検索・ソート等の操作が効率的に行われるための目次・索引のようなものです。 インデックスがなければ、MongoDBはどんなに簡単な検索でも、*collection scan*と呼ばれる操作を行わなければなりません。つまり、コレクション内のすべてのドキュメントをスキャンして、検索に一致するドキュメントのみを選択し、検索結果を返すことになります。 検索に適切なインデックスが存在する場合、MongoDBはそのインデックスを使用して、読み込み対象とするドキュメント数を制限することができます。

RDBMSもそうですが、MongoDBではインデックスは内部的に[B-Tree](https://ja.wikipedia.org/wiki/B木)というデータ構造を使っており、探索時間は$O(\log{}n)$です。単純なcollection scanの探索時間は$O(n)$なので、一般的にインデックスを用いた方が検索時間を圧倒的に早くします。ただし、[インデックスの注意点](#インデックスの注意点)で述べる例外もあるので、注意が必要です。

#### インデックスの確認

インデックスはコレクション毎に定義されていますが、これまで使ってきた`tweets`コレクション内のインデックスを確認するには[`index_information()`](http://api.mongodb.com/python/current/api/pymongo/collection.html?highlight=index#pymongo.collection.Collection.index_information)を使います。

In [ ]:
db.tweets.index_information()

まだ明示的にインデックスを作成していないので、コレクションにあるインデックスは自動的に作成される`_id`に対するインデックスのみです。上記の結果では、`name`はインデックスの名前、`v`はバージョン、`ns`はネームスペース（一般的に`<database>.<collection>`）、`key`はインデックスに含まれるフィールド名を表しています。

インデックスはコレクションのデータと一緒に保存されるので、コレクションをdropしたら(`db.collection.drop()`)そのコレクションに付随されるインデックスも一緒に削除されます。また、ドキュメントを挿入・更新・削除したりするたびに、コレクションにあるインデックスも更新されます。各コレクションは最大64インデックスまで持つことが可能です。

#### 検索処理の詳細

検索やソートがどのインデックスをどうやって使っているかを確認するにはカーソルメソッドの[`explain()`](https://docs.mongodb.com/manual/reference/method/cursor.explain/)を使います。早速使ってみましょう。

In [ ]:
# zhは中国語
db.tweets.find({'lang': 'zh'}).explain()

環境によって実行結果は少し変わるかもしれないので、事前に準備した以下のものを使って説明します（前セルの実行結果と似ているはずです）。

検索を行うとき、MongoDBはコレクションにあるインデックスを見て、いくつかの検索戦略を立てます（上記ドキュメントの`queryPlanner`）。それらを検証し、最もいい戦略`winningPlan`を選び、実行します。今回の場合は使えるインデックスがなかったので、最もいい戦略はcollection scan(`COLLSCAN`)を行うことです。

実行された検索の詳細は`executionStats`にまとめられています。それを見ると、実行時間`executionTimeMillis`が29ミリ秒、スキャンされたドキュメント数`totalDocsExamined`3855（すなわち、コレクションの全ドキュメント）であることが分かります。

今回は取り扱っているのは数MB程度のデータセットなので、インデックスの必要性は低いと言えるかもしれませんが、例えばデータが10GB程度になれば、インデックスなしだと上記の検索が**数十秒**かかると考えられます。MongoDBは固定スキーマがないなど、比較的自由な開発が可能ですが、その反面、事前にしっかりとしたインデックス設計を行わないと期待するパフォーマンスを得ることが難しくなってしまいます。

>**注意**: MongoDBは一旦collection scanを行うと、次の操作を高速化するために、スキャンされたドキュメントをメモリ上に残してしまいます。したがって、上記の`find`/`explain`をもう一度実行すれば、実行時間`executionTimeMillis`が0もしくはそれに近い値になると考えられます。パフォーマンスを検証する際は検索対象データが**メモリに載っていない**ことを保障するために、適宜に`mongo`インスタンスを再起動したり、データベースをドロップして再構築したりしてください（[⇗10.3.5節](#データベース再構築)）。

#### インデックスの作成

それでは先程の検索をより効率的にするために、[`create_index`](http://api.mongodb.com/python/current/api/pymongo/collection.html#pymongo.collection.Collection.create_index)でインデックスを作成しましょう。

In [ ]:
db.tweets.create_index('lang')

ここでは、`lang`フィールドに対するSingleインデックスを作成しています。[ソート](#10.3.2.1-カーソル操作)はデフォルトで昇順になりますが、明示的に示すことも可能です。ソートと同様に`1`なら昇順、`-1`なら降順となります。

In [ ]:
db.tweets.create_index([('lang', pymongo.ASCENDING)])

もう一度`index_information`をコールすると、新しいインデックスが追加されていることがわかります。なお、上記のセルのようにすでに存在するインデックスをもう一度作ろうとしても何もエラー発生せずに無視されます。

In [ ]:
db.tweets.index_information()

インデックスの作成はコレクションに入れる前でも、入れた後でも可能です。しかし、インデックスがあるコレクションに新しいドキュメントを挿入する際には、すべてのインデックスを更新しなければならないため、インデックスがある分だけ挿入が少し遅くなります。また、今回の`tweets`コレクションは小さいので、瞬時にインデックスが作成されましたが、大きなコレクションのインデックスを作成する場合は時間がかかるため、インデックス作成している間にコレクションを、他のスレッド・プロセスからもアクセス可能に保つ必要がある場合は`background`オプションを使うといいでしょう（[⇗公式ドキュメンテーション](https://docs.mongodb.com/manual/reference/method/db.collection.createIndex/#db.collection.createIndex)）。

それでは、先程の検索をもう一回実行してみましょう（**注意**: 実行前には先程の注意欄のことを行ってください）。

In [ ]:
db.tweets.find({'lang': 'zh'}).explain()

ここでも、環境によって実行結果は少し変わるかもしれないので、事前に準備した以下のものを使って説明します。

今回は`queryPlanner`を見てみると、`winningPlan`がいきなり`COLLSCAN`ではなく、`IXSCAN`すなわち、インデックスを使ったスキャンを行っていることがわかります。また、`executionStats`をみると、実行時間が前回の29ミリ秒から3ミリ秒に減っており、またスキャンされたドキュメント数`totalDocsExamined`は前回の3855から50に減っていることが分かります。

今回は最も単純なSingleインデックスを使いましたが、MongoDBには以下の6種類のインデックスがあります。

<p id="table8" style="text-align:center">**表8: インデックスの種類**</p>

| インデックス | 説明 |
| :----- |:-----------|
|[Single](https://docs.mongodb.com/manual/core/index-single/)|一つだけのフィールドに対するインデックス。<br>トップレベルだけでなく、埋め込みドキュメント内のフィールドもインデックス可能です。|
|[Compound](https://docs.mongodb.com/manual/core/index-compound/)|複数のフィールドに対するインデックス。<br>順序があるため、`{a: 1, b: 1}`と`{b: 1, a: 1}`ではインデックスの構造が異なります。|
|[Multikey](https://docs.mongodb.com/manual/core/index-multikey/)|配列の要素に対するインデックス。|
|[Geospatial](https://docs.mongodb.com/manual/applications/geospatial-indexes/)	|[GeoJSON](http://geojson.org/)等の地理空間データを含むフィールドに対するインデックス。<br>`$near`等の[地理空間用のクエリ演算子](https://docs.mongodb.com/manual/reference/operator/query-geospatial/)を使った検索のときに使われます。|
|[Text](https://docs.mongodb.com/manual/core/index-text/)|自然言語に対する全文検索用インデックス。<br>2018年1月現在は日本語には[対応していません](https://docs.mongodb.com/manual/reference/text-search-languages/#text-search-languages)。|
|[Hased](https://docs.mongodb.com/manual/core/index-hashed/)|指定されたフィールドの値のハッシュ値を保存するインデックス。<br>主に[シャーディング](https://docs.mongodb.com/manual/sharding/)環境において、各クラスタの負荷を均一化するために使われています。|

ここで、Multikeyインデックスの例として、各ツイート内のハッシュタグに対するインデックスを作りましょう。

In [ ]:
db.tweets.create_index('entities.hashtags.text')

上記のインデックスがSingleインデックスに見えるかもしれませんが、各ツイートの`hashtags`フィールドが配列であるため、Multikeyインデックスと分類されます。Singleインデックスでは、コレクション内の各ドキュメントはインデックスされたフィールドの値と1対1の関係にあるのに対し、Multikeyインデックスでは各ドキュメントはインデックスされたフィールドの値と1対$n$の関係にあります。今回のデータセットで説明すると、一つのツイートが同時に日本語と英語(すなわち、`lang`が**同時に**`ja`かつ`en`）であることは不可能ですが、同じツイートが複数のハッシュタグを含むことは可能です。

配列要素を含むCompoundインデックス（すなわち、Compound兼Multikeyインデックス）を作成する際、配列に対するフィールドは一つまでと制限されています。それはインデックス自体のエントリー数が爆発的に増えないためです。

それでは、「Javascript」というハッシュタッグを含むフランス語のツイートを検索しましょう。|

In [ ]:
cursor = db.tweets.find({'entities.hashtags.text': 'Javascript', 'lang': 'fr'},
                        {'entities.hashtags': 1, 'text': 1, 'lang': 1})
list(cursor)

ここでは省略しますが、`explain`を使えば、MongoDBが2つのインデックス(`lang`に対するSingleインデックスと`entities.hashtags.text`に対するMultikeyインデックス）を使うクエリプランが立てられていることがわかります。また実際実行された検索では、`totalDocsExamined`が6つであることも確認できます。

>**注意**: ネット上等でMongoDBは一つの検索・ソートでは基本的に一つのインデックスしか使えないという記述が多く見られますが、v2.6からは複数のインデックスの使用([Index Intersection](https://docs.mongodb.com/manual/core/index-intersection/))が可能になりました。


**<練習問題3>**

上記に検索に対してして`explain`を使って、実際どのような検索が行われているかを調べてください。

In [ ]:
## ここでコードを書いたり、セルを追加したりしてください

#### インデックスの属性

インデックス作成時には、インデックスの挙動を変更するためのいくつかのオプションがあります。それらを以下の表にまとめます:

<p id="table9" style="text-align:center">**表9: インデックスの属性**</p>

| 属性 | 説明 |
| :----- |:-----------|
|[Unique](https://docs.mongodb.com/manual/core/index-unique/)|フィールドの値のユニーク性を保障する。<br>重複値の挿入を不可にする。|
|[Sparse](https://docs.mongodb.com/manual/core/index-sparse/) |対象フィールドを持っているドキュメントのみをインデックスする。<br>Uniqueと同時に使用可能で、インデックスの大きさを抑えるメリットがある。|
|[Partial](https://docs.mongodb.com/manual/core/index-partial/) |特定のフィールドを持っているドキュメントのみをインデックスする。<br>新しくv3.2に導入され、Sparseインデックスの機能を拡張したもの。<br>現在、SparseよりPartialの使用が推奨される。|
|[TTL](https://docs.mongodb.com/manual/core/index-ttl/)|特定の経過時間もしくは時刻にドキュメントを自動的に削除する。<br>ログデータ等の処理に便利。|

Uniqueインデックスの例の一つは自動的に作成される`_id`に対するインデックスです。なお、重複の値が存在する状態でUniqueインデックスを作ろうとしたら、エラーが出るので、Uniqueインデックスを使用する際は**挿入前**にインデックスを作ることを勧めます。以前は重複値をドロップすることが可能でしたが、v3.0以降それが不可能になっているので、注意が必要です。

**<練習問題4>**

すでに存在するドキュメント（ツイート）の`ObjectId`を使った新しいドキュメントを`tweets`コレクションに挿入してみて、`_id`に対するインデックスがUniqueであること(すなわち`DuplicateKeyError`が発生すること)を確認してください。

In [ ]:
## ここでコードを書いたり、セルを追加したりしてください

#### インデックスの注意点

インデックスは、取り扱っているコレクションの小さなサブセットを取得するときに最も効果的です。また、クエリによってはインデックスを使わない方が高速だったりします。一般的に、一つのクエリで取得するコレクションの割合が高ければ高いほど、インデックスの効率が低下します。

インデックスを使うということは、2つの参照を行うことを意味します。一つ目はインデックス上のドキュメントへのポインタの参照で、二つ目はそのポインタからドキュメント自体への参照です。例えば、コレクション内のすべてのドキュメントを取得するようなインデックスを使ったクエリを考えましょう。そのクエリ実行にかかる時間は、collection scanに比べて、先程の一つ目の参照の分だけ増えるので、実はインデックスを使わない方が早いです。

インデックスが性能向上につながるかどうかはドキュメント数、ドキュメントサイズ、検索の種類等、様々な要因に依存します。経験則から、検索クエリがコレクションの30％以上を返している場合は、collection scanの方が早い可能性が高くなるので、しっかり検索のパフォーマンスを調べることが大事になってきます。

#### インデックスのまとめ

インデックスは奥が深く、データベースのパフォーマンスに直結します。また、インデックスの各種類・属性を正しく使うことで、本来クライアントサイドで実装しなければならない機能をデータベースサイドでできてしまったりします（例えば、TTLインデックスを使ったログ収集、Geospatialインデックスを作ったドキュメント間の距離を測定するなど）。

小さなデートセットならインデックスのことをあまり心配せずに済むかもしれませんが、大量のデータを扱う場合に検索・ソート性能を向上させるには、自分のデータを理解し、適切なインデックス設計を行うことが大変重要です。

### 10.4.2 MongoDBの統計・メットリックス

MongoDBを使って、たくさんのデータを書き込み・読み出したり、インデックスを作ったりしていると、データベースへの負荷や、コレクション・ドキュメント・インデックスがどれぐらいの容量を使っているかが気になります。

MongoDBにはそれらの情報を調べるためのMongoシェルコマンドやCLIツールがいくつか用意されています。

<p id="table10" style="text-align:center">**表10: 統計・メトリックス対応表**</p>

| Mongoシェル        | PyMongo             | 説明  | 
| ----------------- | ------------------- | ----- |
|`db.serverStatus()`|`db.command('serverStatus')`|MongoDBに関する[多く](https://docs.mongodb.com/manual/reference/command/serverStatus/)のメットリックスを<br>まとめたドキュメントを返します。|
|`db.stats()`       |`db.command('dbstats')`    |選択中のデータベースのコレクション数、<br>インデックス数、容量等の基本データを返します。|
|`db.collection.stats()`|`db.command('collstats', 'collection')`|当該コレクションに関する[多く](https://docs.mongodb.com/manual/reference/command/collStats/#output)のメットリックスを<br>まとめたドキュメントを返します。|

上記はadmin側面が強いというのもあって、PyMongoではそれらに相当するメソッドがありませんが、`db.command`メッソドを使うことで、MongoDBに生をコマンドを送るこができます。

試しに`db.stats()`を実行してみると、以下のような結果になります。

In [ ]:
db.command('dbstats')

#### **<練習問題5>**

上記に紹介したコマンドを実行し、どのような出力なのかを見てみてください。

In [ ]:
## ここでコードを書いたり、セルを追加したりしてください


#### CLIツール
シェルで実行するコマンド以外に、`mongostat`と`mongotop`というCLIツールがあります。

- **`mongostat`**: MongoDBのリアルタイムパフォーマンスデータ（コネクション数、挿入数、検索数等々）を返します。 データベースの負荷が高いと思われるとき等に便利です。 

- **`mongotop`**: MongoDBがコレクション毎に各CRUD操作（挿入、検索、更新、削除）に使っている時間を1秒毎に返します。UNIX系OSの`top`や`htop`に似ています。

MongoDBのメットリクス等に興味がある読者は[公式ドキュメンテーション](https://docs.mongodb.com/manual/administration/monitoring/)を参考にしてください。

### 10.4.3 データのバックアップ・エクスポート

MongoDBのデータをファイルにバックアップ・エクスポートするCLIツールは以下のようなものがあります。

<p id="table11" style="text-align:center">**表11: バックアップ用CLIツール**</p>

| ツール | 説明 |
| :----- |:-----------|
| [`mongoexport`](https://docs.mongodb.com/manual/reference/program/mongoexport/)| テキスト形式(JSON/CSV)のエクスポート用
| [`mongoimport`](https://docs.mongodb.com/manual/reference/program/mongoimport/)| テキスト形式(JSON/CSV)のインポート用
| [`mongodump`](https://docs.mongodb.com/manual/reference/program/mongodump/)| バイナリ形式(BSON)のエクスポート用
| [`mongorestore`](https://docs.mongodb.com/manual/reference/program/mongorestore/)| バイナリ形式(BSON)のインポート用

基本的な使い方は以下の通りです。

- バイナリデータエクスポート
```
mongodump --db <データベース名> --collection <コレクション名> <ディレクトリ>
```

- バイナリデータインポート
```
mongorestore --db <データベース名> --collection <コレクション名> <ディレクトリ>
```

詳しい使い方に関しては表9にリンクを参考にしてください。また、他のバックアップ方法については[公式ドキュメンテーション](https://docs.mongodb.com/manual/core/backups/)を参考にしてください。

## 10.5 総合問題

### 10.5.1 基本操作の復習
1. 各ツイートのユーザ位置情報に「Japan」もしくは「日本」という文字列が含まれていてかつ言語が英語であるものを検索し、該当ツイートの本文および`ObjectId`のみを取得せよ。
2. 各ツイートのユーザ位置情報を言語毎の集合にグループ化せよ。（ヒント: `$project`、`$unwind`、`$addToSet`を使用せよ。）

In [ ]:
## ここでコードを書いたり、セルを追加したりしてください

### 10.5.2 Geospatialインデックスの応用

これまで使ってきたTwitterデータのほとんどに位置情報が付いています。その位置情報は文字列ですが、[Google Maps Geocoding API](https://developers.google.com/maps/documentation/geocoding/start)等を使って国・都市の名前から緯度・経度を取得することが可能です。(1)その緯度・経度情報を取得し、(2)各ドキュメントに新しいフィールドとして緯度・経度情報を追加した上で、(3)そのフィールドに対してGeospatialインデックスの一種である[2dsphere](https://docs.mongodb.com/manual/core/2dsphere/)を作ってください。その上で、(4)[`$near`](https://docs.mongodb.com/manual/reference/operator/query/near/)演算子を使って東京から5000km以上15000km未満の距離にあるユーザのツイートを取得せよ。

- **注1**: 位置データが無効もしくは存在しないツイートは無視してください。
- **注2**: APIコールには[`requests`](http://docs.python-requests.org/en/master/)、API結果の処理には[`json`](https://docs.python.org/3/library/json.html)が便利でしょう。

In [ ]:
## ここでコードを書いたり、セルを追加したりしてください